<a href="https://colab.research.google.com/github/yutaro-tanaka-yt2705/genome_colab/blob/main/file_prep_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 0. Prepare general packages

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import glob
from tqdm import tqdm

In [ ]:
#make general folder in google drive
output_folder = '' #folder in google drive to save fastq file.
!mkdir $output_folder

## 1. Download files

Obtain files from EBI (European Nucleotide Archive).

https://sra-explorer.info/#

More information : https://ena-docs.readthedocs.io/en/latest/retrieval/file-download.html

In [ ]:
file_url = '' #example. replace accordingly.
file_name = file_url.replace('ftp://ftp.sra.ebi.ac.uk', '')

In [ ]:
!mkdir $output_folder/raw_fastq/

In [ ]:
!curl -L $file_url -o $file_name
!mv /content/$file_name $output_folder/$file_name

In [ ]:
#download GRCh38 (for humans) or other reference genome
ref_genome_url = '' #NCBI url for reference genome
ref_genome_path = output_folder / 
!curl -L $ref_genome_url -o $ref_genome_path

## 2. Perform Quality Control.



In [ ]:
!mkdir qc
!cd /content/qc/
# Install trim galore(perl)
!wget https://raw.githubusercontent.com/FelixKrueger/TrimGalore/master/trim_galore -P /content/qc/
!chmod a+x /content/qc/trim_galore
!/content/qc/trim_galore

# Install cutadapt(python)
!pip3 install cutadapt -q
# Install pigz
!sudo apt install pigz

#make output folder for trimmed fastq files.
!mkdir $output_folder/trimmed_fastq/

In [ ]:
#get list of all input ids.
rawfastqfiles = glob.glob(output_folder+'/raw_fastq/')
print(len(rawfastqfiles))
print(rawfastqfiles)

In [ ]:
#check if any trimmed files exist.
trimmedfastqfiles = glob.glob(output_folder+'/trimmed_fastq/')
print(len(trimmedfastqfiles))
print(trimmedfastqfiles)

In [ ]:
indir, outdir = 
file1, file2 = '_1.fastq.gz', '_2.fastq.gz'
for id in ids:
  id = id.replace(indir, '').replace(file1, '')
  outfile = outdir+id+'_1_val_1.fq.gz'
  if outfile not in done_ids:
    input_file_1, input_file_2 = indir+id+file1, indir+id+file2
    command = ('/content/clean/trim_galore -q 25 --phred33 --length 36 --illumina --paired --stringency 3 -j 2 '+ \
    str(input_file_1) + ' ' + str(input_file_2) + ' '+\
    '-o '+ str(outdir))
    !$command

## 3. Perform Alignment

In [ ]:
#install MiniMap2
!git clone https://github.com/lh3/minimap2
!cd minimap2 && make

In [ ]:
#make alignment index file
!/content/minimap2/minimap2 -d \
'/content/drive/MyDrive/015_Ciccia_Lab/immunotherapeutics_project/GWAS/minimap_ref.mmi' \ $ref_genome_path

In [ ]:
trimmed_fastqs = glob.glob('/content/drive/MyDrive/015_Ciccia_Lab/immunotherapeutics_project/GWAS/qc_hugo_2/*_1.fq.gz')
print(len(trimmed_fastqs))
print(trimmed_fastqs)

In [ ]:
aligned_ids = glob.glob('/content/drive/MyDrive/015_Ciccia_Lab/immunotherapeutics_project/GWAS/minimap2/*')
print(len(aligned_ids))
print(aligned_ids)

In [ ]:
ref=''
trimmedfastqdir='' #trimmed fastq file folder
alignedsamdir='' #aligned sam file folder.
file_1, file_2 = '_1_val_1.fq.gz', '_2_val_2.fq.gz'

for id in ids:
  select_id = id.replace(trimmedfastqdir, '').replace(file_1, '')
  done_id = alignedsamdir + select_id + '.sam'
  if done_id not in done_ids:
      command = '/content/minimap2/minimap2 -a '+ref+' '+id+' > '+done_id
      !$command

## 4. Convert sam to bam file.

In [ ]:
!sudo apt install samtools -q

In [ ]:
alignedsamfiles = glob.glob(alignedsamdir+'*')
print(len(alignedsamfiles)))
print(alignedsamfiles)

In [ ]:
bam_dir = output_folder + '/bam_files/'
bam_files = glob.glob(bam_dir+'*')
print(len(bam_files))
print(bam_files)

In [ ]:
#convert sam files into bam files.
for file in tqdm(alignedsamfiles):
  select_id = file.replace(inputdir, '').replace('.sam', '')
  done_id = bam_dir + select_id + '.bam'
  if file not in bam_files:
      command = 'samtools view -S -b '+file+' > '+done_id
      !$command

In [ ]:
sortedbamdir = output_folder + '/sortedbam_files/'
sortedbam_files = glob.glob(sortedbamdir+'*')
print(len(sortedbam_files))
print(sortedbam_files)

In [ ]:
#sort bam files
bam_files = glob.glob(bam_dir+'*')
for file in tqdm(bam_files):
  select_id = file.replace(bam_dir, '')
  done_id = sortedbamdir + select_id
  if file not in sortedbam_files:
    command = 'samtools sort '+file+' -o '+done_id
    !$command

In [ ]:
#call genome alignment
!samtools faidx $raw_genome_path